In [65]:
input_file = "input.txt"

In [66]:
NEO4J_URI = "neo4j://localhost:7687"
NEO4J_USER = "neo4j"
NEO4J_PASSWORD = "pierre!!"

In [67]:
from graphdatascience import GraphDataScience
gds = GraphDataScience(NEO4J_URI, auth=(NEO4J_USER, NEO4J_PASSWORD))

# Check the installed GDS version on the server
print(gds.version())
assert gds.version()

2.5.5


In [68]:
lines = open(input_file, 'r').readlines()
dir_vects = {'R':{'x':1,'y':0}, 'L':{'x':-1,'y':0}, 'U':{'x':0,'y':1}, 'D':{'x':0,'y':-1}}
vects = [({z:dir_vects[dir][z]*int(length) for z in dir_vects[dir]}, color[1:-1]) for dir, length, color in [l.strip().split(' ') for l in lines]]

In [69]:
nodes = [{'x':0,'y':0}]
edges = []
for v in vects:
    nodes.append({'x': nodes[-1]['x'] + v[0]['x'], 'y':nodes[-1]['y'] + v[0]['y']})
    edges.append((nodes[-2], nodes[-1], v[1]))


In [70]:
gds.run_cypher("""CREATE INDEX cube_x_y IF NOT EXISTS
FOR (c:Point) ON (c.x, c.y)""")

""


In [71]:
gds.run_cypher("""
UNWIND $nodes AS n
MERGE (c:Cube:Point {x:n.x, y:n.y})
""", {'nodes':nodes})

""


In [72]:
gds.run_cypher("""
UNWIND $edges AS e
MATCH (c1:Cube:Point {x:e[0].x, y:e[0].y})
MATCH (c2:Cube:Point {x:e[1].x, y:e[1].y})
CREATE (c1)-[:DIG_TO {color:e[2]}]->(c2)
""", {'edges':edges})

""


In [73]:

gds.run_cypher("""
MATCH (c:Cube)
WITH min(c.x) AS minx, max(c.x) AS maxx, min(c.y) AS miny, max(c.y) AS maxy
UNWIND [x IN range(2*minx - 1, 2*maxx + 1)| 0.5 * x] AS x
UNWIND [y IN range(2*miny - 1, 2*maxy + 1)| 0.5 * y] AS y
CALL {
WITH x, y
MERGE (:Point {x:x, y:y})
} IN TRANSACTIONS OF 10000 ROWS
""")

""


In [74]:
gds.run_cypher("""
MATCH (p:Point)
WHERE p.x%1 = 0 AND p.y%1 = 0
SET p:Cube
""")

gds.run_cypher("""
MATCH (p:Point)
WITH p, p.x AS x, p.y AS y
ORDER BY x, y 
WITH collect(p) AS col, x
CALL apoc.nodes.link(col, "UP")
""")
               
gds.run_cypher("""
MATCH (p:Point)
WITH p, p.x AS x, p.y AS y
ORDER BY y, x 
WITH collect(p) AS col, y
CALL apoc.nodes.link(col, "RIGHT")
""")

gds.run_cypher("""
MATCH (a)-[e:DIG_TO]->(b)
MATCH p=shortestPath((a)-[:UP|RIGHT*]-(b))
UNWIND nodes(p) AS t
SET t:Trench
""")

""


In [75]:
gds.run_cypher("""
MATCH (source:Point&!Trench)
OPTIONAL MATCH (source)-[r:UP|RIGHT]->(target:Point&!Border)
WITH gds.graph.project('regions', source, target) AS g
RETURN
  g.graphName AS graph, g.nodeCount AS nodes, g.relationshipCount AS rels
""")

,graph,nodes,rels
0,regions,374442,734566


In [76]:
G = gds.graph.get("regions")
gds.wcc.write(G, writeProperty="region")

writeMillis                                                            997
nodePropertiesWritten                                               374442
componentCount                                                           2
componentDistribution    {'min': 137180, 'p5': 137180, 'max': 237262, '...
postProcessingMillis                                                    31
preProcessingMillis                                                      0
computeMillis                                                           60
configuration            {'writeProperty': 'region', 'jobId': '366548a2...
Name: 0, dtype: object

In [78]:
gds.run_cypher("""
MATCH (p:Point)
WITH min(p.x) AS minx, min(p.y) AS miny
MATCH (ext:Point {x:minx, y:miny})
WITH ext.region AS ext_region
MATCH (c:Cube)
WHERE c:Trench OR c.region <> ext_region
RETURN count(c) AS part1
""")

,part1
0,35991
